# Tweets from bioRxiv and medRxiv

Find and display events from the CSHL DOI prefix 10.1101 from the last 7 days, and find the most tweeted preprints.

In [11]:
import sys
sys.path.append('../')
sys.path.insert(0, '..')

import pandas as pd # data analysis library
import json
import mrced2 # module to run event data queries
import os # some file manipulations
import math # some number manipulations
import altair.vegalite.v3 as alt # some data visualizations
from IPython.display import Markdown as md # some markdown manipulations
from datetime import datetime, date, timedelta # some date manipulations

In [12]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = (date.today() - timedelta(days = 7)).strftime('%Y-%m-%d')
end_date = date.today()

In [13]:
ed = mrced2.eventData(mailto = email)
ed.buildQuery({'obj-id.prefix' : prefix, 'source': 'twitter', 'rows': 500,'from-occurred-date' : start_date, 'until-occurred-date' : end_date})

https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&obj-id.prefix=10.1101&source=twitter&rows=500&from-occurred-date=2021-09-30&until-occurred-date=2021-10-07


In [14]:
ed.runQuery(retry = 5)

Event Data query started...
API query complete  200
output file written to 1101/tweets.json


In [15]:
pages = math.ceil(ed.events.getHits() / 1000)

9939 events found


In [16]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = date.today() - timedelta(days = 7)
end_date = date.today()

# find the all result pages for the search
ed = mrced2.eventData(mailto = email)
ed.getAllPages(pages, {'rows': 1000, 'obj-id.prefix' : prefix, 'source': 'twitter', 'from-occurred-date' : start_date, 'until-occurred-date' : end_date}, fileprefix = '1101/tweets_') 

https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-09-30&until-occurred-date=2021-10-07
Event Data query started...
API query complete  200
output file written to 1101/tweets_0000.json
https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&cursor=7ca577f7-c6ef-43c0-a2e6-d72a2d69238f&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-09-30&until-occurred-date=2021-10-07
Event Data query started...
API query complete  200
output file written to 1101/tweets_0001.json
https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&cursor=bb9b11c1-0e5a-43fc-a144-60eaddaae78d&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-09-30&until-occurred-date=2021-10-07
Event Data query started...
API query complete  200
output file written to 1101/tweets_0002.json
https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&curso

Initialisation to look at the properties of the results.

In [17]:
jd1 = mrced2.eventRecord() # instance of a class to interpret the events
files = os.listdir('1101') # get all the filenames

jd1.mergeJsons(files, folder = '1101') # load the json event data from multiple files

failed to load preprint_tweets_2021-07-29.csv
failed to load preprint_tweets_2021-09-06.csv
failed to load preprint_tweets_2021-09-13.csv
failed to load .DS_Store
failed to load .gitkeep
failed to load preprint_tweets_2021-08-30.csv
failed to load preprint_tweets_2021-08-09.csv
failed to load preprint_tweets_2021-08-23.csv
failed to load preprint_tweets_2021-08-02.csv
failed to load preprint_tweets_2021-08-16.csv
failed to load .ipynb_checkpoints
failed to load preprint_tweets_2021-10-05.csv
failed to load preprint_tweets_2021-10-04.csv
failed to load preprint_tweets_2021-10-06.csv
failed to load preprint_tweets_2021-07-26.csv
failed to load preprint_tweets_2021-09-20.csv
output file written to 1101/tweets.json


In [18]:
js = json.load(open("1101/tweets.json"))
df = pd.json_normalize(js, record_path = ['message', 'events'])
gdf = df.groupby(['obj_id']).size().reset_index(name='count').sort_values('count', ascending=False)
cdf = gdf[gdf['count'] >= 3]

In [19]:
email = "info@front-matter.io"
rest = mrced2.restApi(email = email)

data = []
for index, row in cdf.iterrows():
    rest.runQuery(row)
    if rest.work is not None and date.fromisoformat(rest.work["posted"]) >= start_date:
        data.append(rest.work)
print(data)
    
tdf = pd.DataFrame(data, columns=['doi','tweets','archive','subject-area','covid','title','authors','abstract','posted'])
tdf.to_csv('1101/preprint_tweets_' + str(date.today().strftime('%Y-%m-%d')) + '.csv')

tdf.head(50)

REST API query started for 10.1101/2021.07.13.452194...
REST API query complete  200
REST API query started for 10.1101/2021.07.31.21261387...
REST API query complete  200
REST API query started for 10.1101/2021.09.28.21264262...
REST API query complete  200
REST API query started for 10.1101/2021.09.28.21264260...
REST API query complete  200
REST API query started for 10.1101/2021.09.22.21263977...
REST API query complete  200
REST API query started for 10.1101/2021.04.19.21255739...
REST API query complete  200
REST API query started for 10.1101/2021.07.08.21260210...
REST API query complete  200
REST API query started for 10.1101/2021.08.24.21262415...
REST API query complete  200
REST API query started for 10.1101/2021.06.01.21258176...
REST API query complete  200
REST API query started for 10.1101/2021.09.30.462488...
REST API query complete  200
REST API query started for 10.1101/2021.04.20.21255670...
REST API query complete  200
REST API query started for 10.1101/2021.08.19.2

,doi,tweets,archive,subject-area,covid,title,authors,abstract,posted
0,10.1101/2021.09.30.462488,423,bioRxiv,Immunology,True,Durability of immune responses to the BNT162b2...,"[{'name': 'Mehul S. Suthar'}, {'name': 'Prabhu...",<p>The development of the highly efficacious m...,2021-09-30
1,10.1101/2021.09.29.21264325,220,medRxiv,Epidemiology,True,Assessing the Burden of COVID-19 in Developing...,"[{'name': 'Andrew Levin'}, {'name': 'Nana Owus...",<sec><title>Introduction</title><p>The infecti...,2021-10-01
2,10.1101/2021.09.30.21264363,197,medRxiv,Infectious Diseases (except HIV/AIDS),True,SARS-CoV-2 infections elicit higher levels of ...,"[{'name': 'Elizabeth M. Anderson'}, {'name': '...",<p>Severe acute respiratory syndrome coronavir...,2021-10-01
3,10.1101/2021.09.29.21264272,145,medRxiv,Infectious Diseases (except HIV/AIDS),True,Associations between SARS-CoV-2 variants and r...,"[{'name': 'Miguel I. Paredes'}, {'name': 'Step...",<sec><title>Background</title><p>The COVID-19 ...,2021-09-30
4,10.1101/2021.09.28.462148,83,bioRxiv,Neuroscience,False,Adeno-associated virus (AAV) reduces cortical ...,"[{'name': 'Christos M. Suriano'}, {'name': 'Je...",<p>Recombinant adeno-associated viruses (AAVs)...,2021-09-30
5,10.1101/2021.09.30.462231,75,bioRxiv,Biochemistry,False,Structures of core eukaryotic protein complexes,"[{'name': 'Ian R. Humphreys'}, {'name': 'Jimin...",<p>Protein-protein interactions play critical ...,2021-09-30
6,10.1101/2021.09.30.21264376,42,medRxiv,Public and Global Health,True,Estimates of pandemic excess mortality in Indi...,"[{'name': 'Murad Banaji'}, {'name': 'Aashish G...",<sec><title>Background</title><p>The COVID-19 ...,2021-10-01
7,10.1101/2021.09.28.21263801,30,medRxiv,Epidemiology,True,Analysis of overdispersion in airborne transmi...,"[{'name': 'Swetaprovo Chaudhuri'}, {'name': 'P...",<p>Superspreading events and overdispersion ar...,2021-09-30
8,10.1101/2021.09.30.462538,22,bioRxiv,Biophysics,False,New tools for automated cryo-EM single-particl...,"[{'name': 'Dari Kimanius'}, {'name': 'Liyi Don...",<label>1</label><title>Abstract</title><p>We d...,2021-09-30
9,10.1101/2021.09.30.462505,21,bioRxiv,Genetics,False,S. pombe wtf genes use dual transcriptional re...,"[{'name': 'Nicole L. Nuckolls'}, {'name': 'Ana...",<p>Meiotic drivers bias gametogenesis to ensur...,2021-10-01


### Tweets of bioRxiv and medRxiv preprints

In [20]:
num_rows = tdf['archive'].count()
num_covid = tdf['covid'].value_counts(ascending=True)[1]
num_biorxiv = tdf['archive'].value_counts(ascending=True)[1]
num_medrxiv = tdf['archive'].value_counts(ascending=True)[0]
end_date = date.today().strftime('%Y-%m-%d')
max_count = tdf['tweets'].max()

md('{} preprints (including {} covering SARS-CoV-2, {} from bioRxiv and {} from medRxiv) published in the last 7 days before {} had been tweeted at least 3 times (maximum {}).'.format(num_rows, num_covid, num_biorxiv, num_medrxiv, end_date, max_count))

92 preprints (including 17 covering SARS-CoV-2, 77 from bioRxiv and 15 from medRxiv) published in the last 7 days before 2021-10-07 had been tweeted at least 3 times (maximum 423).